In [58]:
import pandas as pd

In [59]:
data = pd.read_csv('data/top_100_movies_full_best_effort.csv')
data

,Rank,Title,Year,Genre(s),Director,Main Actor(s),Country,IMDb Rating,Rotten Tomatoes %,Runtime (mins),Language,Oscars Won,Box Office ($M),Metacritic Score
0,1,The Shawshank Redemption,1994,Drama,Frank Darabont,Tim Robbins|Morgan Freeman,United States,9.3,91.0,142.0,English,0,58.0,82.0
1,2,The Godfather,1972,Crime|Drama,Francis Ford Coppola,Marlon Brando|Al Pacino,United States,9.2,98.0,175.0,English,3,246.1,100.0
2,3,The Dark Knight,2008,Action|Crime|Drama,Christopher Nolan,Christian Bale|Heath Ledger,United States|United Kingdom,9.0,94.0,152.0,English,2,1004.9,84.0
3,4,The Godfather: Part II,1974,Crime|Drama,Francis Ford Coppola,Al Pacino|Robert De Niro,United States,9.0,97.0,202.0,English,6,48.5,90.0
4,5,12 Angry Men,1957,Crime|Drama,Sidney Lumet,Henry Fonda|Lee J. Cobb,United States,9.0,100.0,96.0,English,0,1.0,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,Lawrence of Arabia,1962,Adventure|Biography|Drama,David Lean,Peter O'Toole|Omar Sharif,United Kingdom,8.3,98.0,222.0,English,7,70.0,NaN
96,97,The Bridge on the River Kwai,1957,Adventure|Drama|War,David Lean,William Holden|Alec Guinness,United Kingdom|United States,8.1,93.0,161.0,English,7,30.6,NaN
97,98,Double Indemnity,1944,Crime|Drama|Film-Noir,Billy Wilder,Fred MacMurray|Barbara Stanwyck,United States,8.0,98.0,107.0,English,0,5.7,NaN
98,99,Annie Hall,1977,Comedy|Romance,Woody Allen,Woody Allen|Diane Keaton,United States,8.0,97.0,93.0,English,4,38.3,NaN


# Data Structure Overview

In [60]:
data.shape

(100, 14)

In [61]:
data.columns

Index(['Rank', 'Title', 'Year', 'Genre(s)', 'Director', 'Main Actor(s)',
       'Country', 'IMDb Rating', 'Rotten Tomatoes %', 'Runtime (mins)',
       'Language', 'Oscars Won', 'Box Office ($M)', 'Metacritic Score'],
      dtype='object')

In [62]:
data.dtypes

Rank                   int64
Title                 object
Year                   int64
Genre(s)              object
Director              object
Main Actor(s)         object
Country               object
IMDb Rating          float64
Rotten Tomatoes %    float64
Runtime (mins)       float64
Language              object
Oscars Won             int64
Box Office ($M)      float64
Metacritic Score     float64
dtype: object

In [63]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Rank               100 non-null    int64  
 1   Title              100 non-null    object 
 2   Year               100 non-null    int64  
 3   Genre(s)           100 non-null    object 
 4   Director           100 non-null    object 
 5   Main Actor(s)      100 non-null    object 
 6   Country            100 non-null    object 
 7   IMDb Rating        99 non-null     float64
 8   Rotten Tomatoes %  97 non-null     float64
 9   Runtime (mins)     99 non-null     float64
 10  Language           100 non-null    object 
 11  Oscars Won         100 non-null    int64  
 12  Box Office ($M)    83 non-null     float64
 13  Metacritic Score   50 non-null     float64
dtypes: float64(5), int64(3), object(6)
memory usage: 11.1+ KB


# Duplicate data

In [64]:
# 1. Standardize Title Column
# Remove text in parentheses such as (dup), (dup2)
data['title'] = data['Title'].str.replace(r'\s*\([^)]*\)', '', regex=True)

In [65]:
data['title'] = data['title'].str.strip()

In [66]:
data['title'].head(3).tolist()

['The Shawshank Redemption', 'The Godfather', 'The Dark Knight']

In [67]:
# 2. Create a new variable to "inspect" duplicates (don't overwrite the main variable yet)
# keep=False = keep all duplicate rows (both first occurrence and duplicates)
duplicates_data = data[data.duplicated(subset=['title'], keep=False)]

In [68]:
print(duplicates_data.sort_values(by='title')[['Rank', 'title', 'Year']])

    Rank                         title  Year
75    76                Paths of Glory  1957
91    92                Paths of Glory  1957
44    45                      Rashomon  1950
78    79                      Rashomon  1950
72    73  The Bridge on the River Kwai  1957
96    97  The Bridge on the River Kwai  1957
42    43            The Great Dictator  1940
74    75            The Great Dictator  1940
46    47                 The Third Man  1949
70    71                 The Third Man  1949


# Multi-value

In [69]:
# Find which columns contain the pipe '|' delimiter
print("Columns containing '|':")
for column in data.columns:
    # Check if pipe symbol exists
    has_pipe = data[column].astype(str).str.contains('|', regex=False).any()
    if has_pipe:
        print(column)

Columns containing '|':
Genre(s)
Director
Main Actor(s)
Country
Language


In [70]:
print("--- Detailed count of rows with delimiters ---")
for column in data.columns:
    # Count rows containing pipe symbol
    pipe_rows_count = data[column].astype(str).str.contains('|', regex=False).sum()
    if pipe_rows_count > 0:
        print(column, f"Rows with multiple values: {pipe_rows_count} rows")

--- Detailed count of rows with delimiters ---
Genre(s) Rows with multiple values: 89 rows
Director Rows with multiple values: 5 rows
Main Actor(s) Rows with multiple values: 99 rows
Country Rows with multiple values: 27 rows
Language Rows with multiple values: 5 rows


# Missing Data

In [71]:
data.isnull().sum()

Rank                  0
Title                 0
Year                  0
Genre(s)              0
Director              0
Main Actor(s)         0
Country               0
IMDb Rating           1
Rotten Tomatoes %     3
Runtime (mins)        1
Language              0
Oscars Won            0
Box Office ($M)      17
Metacritic Score     50
title                 0
dtype: int64

# Statistical Summary

In [72]:
data.describe()

,Rank,Year,IMDb Rating,Rotten Tomatoes %,Runtime (mins),Oscars Won,Box Office ($M),Metacritic Score
count,100.000000,100.000000,99.000000,97.000000,99.000000,100.000000,83.000000,50.00
mean,50.500000,1977.340000,8.385859,92.670103,130.000000,1.810000,200.619277,82.68
std,29.011492,22.743728,0.305063,6.454523,30.148273,2.372805,270.319883,11.14
min,1.000000,1931.000000,7.700000,71.000000,86.000000,0.000000,0.000000,59.00
25%,25.750000,1957.000000,8.100000,91.000000,108.500000,0.000000,24.800000,75.25
50%,50.500000,1980.000000,8.400000,94.000000,125.000000,1.000000,66.800000,84.50
75%,75.250000,1998.000000,8.600000,97.000000,146.500000,3.000000,279.750000,92.75
max,100.000000,2019.000000,9.300000,100.000000,222.000000,11.000000,1119.900000,100.00


# Rename Columns

In [73]:
data.columns

Index(['Rank', 'Title', 'Year', 'Genre(s)', 'Director', 'Main Actor(s)',
       'Country', 'IMDb Rating', 'Rotten Tomatoes %', 'Runtime (mins)',
       'Language', 'Oscars Won', 'Box Office ($M)', 'Metacritic Score',
       'title'],
      dtype='object')

In [75]:
data = data.rename(
    columns={
        'Rank': 'rank',
        'Title': 'title',
        'Year': 'year',
        'Genre(s)': 'genres',
        'Director': 'director',
        'Main Actor(s)': 'main_actors',
        'Country': 'country',
        'IMDb Rating': 'imdb_rating',
        'Rotten Tomatoes %': 'rotten_tomatoes_pct',
        'Runtime (mins)': 'runtime_mins',
        'Language': 'language',
        'Oscars Won': 'oscars_won',
        'Box Office ($M)': 'box_office_millions',
        'Metacritic Score': 'metacritic_score'
    }
)

In [76]:
data.columns

Index(['rank', 'title', 'year', 'genres', 'director', 'main_actors', 'country',
       'imdb_rating', 'rotten_tomatoes_pct', 'runtime_mins', 'language',
       'oscars_won', 'box_office_millions', 'metacritic_score', 'title'],
      dtype='object')

# Outliers Detection

In [77]:
# Data Validation (Outlier Detection)
print("--- Data Quality Validation ---")

# 1. Validate Year (should not be too old or in the future)
current_year = 2025
invalid_year = data[(data['year'] < 1800) | (data['year'] > current_year)]
print(f"Invalid years: {len(invalid_year)} rows")

# 2. Validate IMDb Rating (must be between 0-10)
invalid_imdb = data[(data['imdb_rating'] < 0) | (data['imdb_rating'] > 10)]
print(f"Invalid IMDb ratings: {len(invalid_imdb)} rows")

# 3. Validate Runtime (should not be negative or zero)
invalid_runtime = data[data['runtime_mins'] <= 0]
print(f"Invalid runtime: {len(invalid_runtime)} rows")

# 4. Validate percentage scores (must be between 0-100)
invalid_rt = data[(data['rotten_tomatoes_pct'] < 0) | (data['rotten_tomatoes_pct'] > 100)]
print(f"Invalid Rotten Tomatoes scores: {len(invalid_rt)} rows")

# 5. Validate Oscar count (should not be negative)
invalid_oscars = data[data['oscars_won'] < 0]
print(f"Invalid Oscar counts: {len(invalid_oscars)} rows")

--- Data Quality Validation ---
Invalid years: 0 rows
Invalid IMDb ratings: 0 rows
Invalid runtime: 0 rows
Invalid Rotten Tomatoes scores: 0 rows
Invalid Oscar counts: 0 rows
